In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline


from orbit.models import DLT
from orbit.diagnostics.metrics import smape, wmape
from orbit.diagnostics.plot import plot_predicted_data
from orbit.utils.simulation import make_trend, make_seasonality, make_regression

In [2]:
SERIES_LEN = 720
SEED = 2022
rw = make_trend(SERIES_LEN, rw_loc=0.05, rw_scale=0.5, seed=SEED)
# add obs noise
rw = (rw + np.random.normal(0, 1, rw.shape)).reshape(-1, 1)
df = pd.DataFrame(rw, columns=['y'])
dt = pd.date_range(start='2016-01-04', periods=SERIES_LEN, freq="1W")
df['ds'] = dt
# plt.plot(rw, 'o')

In [3]:
smooth_inputs = np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.5])

In [7]:
# create a new directory because it does not exist 
fig_dir = './temp'

is_exist = os.path.exists(fig_dir)
if not is_exist:

    os.makedirs(fig_dir)

In [17]:
fig_paths = list()
for idx, smooth_input in enumerate(smooth_inputs):
    dlt = DLT(
        # default
        response_col='y',
        date_col='ds',
        estimator='stan-map',
        level_sm_input=smooth_input,
        slope_sm_input=smooth_input,
        verbose=False,
    )
    dlt.fit(df)
    fig_path = '{}/fig{}.jpg'.format(fig_dir, idx)
    fig_paths.append(fig_path)
    pred = dlt.predict(df)
    plot_predicted_data(
        training_actual_df=df,
        predicted_df=pred,
        date_col='ds',
        actual_col='y',
        lw='3',
        path=fig_path,
        is_visible=False,
        title='smooth param = {}'.format(smooth_input),
        fontsize=24,
    )

In [18]:
import imageio
print(imageio.__version__)
with imageio.get_writer("{}/varying_smooth_inputs.gif".format(fig_dir), mode="I", duration=0.5) as writer:
    for fig_path in fig_paths:
        image = imageio.imread(fig_path)
        writer.append_data(image)

2.16.2
